In [31]:
import pandas as pd

df = pd.read_csv("data/data.csv", delimiter=";", decimal=".")
df.head()

,participant,condition,order,event,target_x,target_y,mean_x,mean_y,dist_x,dist_y,euclidean,sd_x,sd_y,rmss2s
0,10,tobii,0,1,110,90,107.262613,56.427644,12.909590,33.572356,36.527712,14.676260,9.764385,62.156319
1,10,tobii,0,2,960,140,972.443642,138.547800,12.638338,9.265222,17.542150,7.086793,11.231340,68.013904
2,10,tobii,0,3,1810,90,1780.021462,103.984987,30.049228,14.041251,34.267276,9.918404,6.478437,72.550474
3,10,tobii,0,4,510,340,519.052545,320.734202,12.994772,19.332393,26.029599,12.984787,11.426325,75.680898
4,10,tobii,0,5,1410,340,1418.933342,326.705189,9.056418,13.360879,16.924362,4.481371,6.321749,58.972786


In [36]:
# Calculate the mean for dist_x and dist_y for the conditions tobii, tobii_filtered, uniteye, uniteye_filtered
mean_df = df.groupby('condition')[['dist_x', 'dist_y', 'euclidean', 'sd_x', 'sd_y', 'rmss2s']].mean()
mean_df

,dist_x,dist_y,euclidean,sd_x,sd_y,rmss2s
condition,,,,,,
tobii,22.597374,28.425218,39.798871,13.280550,12.928298,90.440579
uniteye,147.608274,141.719177,226.003278,65.625365,64.682136,758.095854
uniteye_filtered,141.788854,133.071396,213.538731,29.345580,27.602975,68.122234


In [37]:
from math import degrees, atan2


def px_to_deg(px: int, px_inch: int, eye_screen_distance_mm: int) -> float:
    return degrees(atan2(px / px_inch * 25.4, eye_screen_distance_mm))


display_size_in_cm = [34.5, 19.5]
pixel_per_inch = 141
pixel_per_cm = pixel_per_inch / 2.54

# Translate the mean values from pixel to cm
mean_df['dist_x_cm'] = mean_df['dist_x'] / pixel_per_cm
mean_df['dist_y_cm'] = mean_df['dist_y'] / pixel_per_cm
mean_df['rmss2s_cm'] = mean_df['rmss2s'] / pixel_per_cm
mean_df['euclidean_cm'] = mean_df['euclidean'] / pixel_per_cm
mean_df['sd_x_cm'] = mean_df['sd_x'] / pixel_per_cm
mean_df['sd_y_cm'] = mean_df['sd_y'] / pixel_per_cm

mean_df['euclidean_vd'] = mean_df['euclidean'].apply(lambda x: px_to_deg(x, pixel_per_inch, 600))
 
mean_df

,dist_x,dist_y,euclidean,sd_x,sd_y,rmss2s,dist_x_cm,dist_y_cm,rmss2s_cm,euclidean_cm,sd_x_cm,sd_y_cm,euclidean_vd
condition,,,,,,,,,,,,,
tobii,22.597374,28.425218,39.798871,13.280550,12.928298,90.440579,0.407073,0.512057,1.629213,0.716944,0.239238,0.232893,0.684599
uniteye,147.608274,141.719177,226.003278,65.625365,64.682136,758.095854,2.659043,2.552955,13.656479,4.071265,1.182187,1.165196,3.881821
uniteye_filtered,141.788854,133.071396,213.538731,29.345580,27.602975,68.122234,2.554211,2.397173,1.227166,3.846726,0.528637,0.497245,3.668332


In [34]:
# Calculate in how many areas the display can be divided with the given mean values
mean_df['areas_size_x'] = 2 * (mean_df['dist_x_cm'] + mean_df['sd_x_cm'])
mean_df['areas_size_y'] = 2 * (mean_df['dist_y_cm'] + mean_df['sd_y_cm'])

mean_df['areas_x'] = display_size_in_cm[0] / mean_df['areas_size_x']
mean_df['areas_y'] = display_size_in_cm[1] / mean_df['areas_size_y']

# round the area values to integers
mean_df['areas_x'] = mean_df['areas_x'].round(0).astype(int)
mean_df['areas_y'] = mean_df['areas_y'].round(0).astype(int)

mean_df['areas_size_x'] = mean_df['areas_size_x'].round(2)
mean_df['areas_size_y'] = mean_df['areas_size_y'].round(2)

# calculate the total number of areas
mean_df['number_of_areas'] = mean_df['areas_x'] * mean_df['areas_y']

mean_df

,dist_x,dist_y,euclidean,sd_x,sd_y,dist_x_cm,dist_y_cm,euclidean_cm,sd_x_cm,sd_y_cm,euclidean_vd,areas_size_x,areas_size_y,areas_x,areas_y,number_of_areas
condition,,,,,,,,,,,,,,,,
tobii,22.597374,28.425218,39.798871,13.280550,12.928298,0.407073,0.512057,0.716944,0.239238,0.232893,0.684599,1.29,1.49,27,13,351
uniteye,147.608274,141.719177,226.003278,65.625365,64.682136,2.659043,2.552955,4.071265,1.182187,1.165196,3.881821,7.68,7.44,4,3,12
uniteye_filtered,141.788854,133.071396,213.538731,29.345580,27.602975,2.554211,2.397173,3.846726,0.528637,0.497245,3.668332,6.17,5.79,6,3,18


In [35]:
mean_df.to_csv("./aois_15inch.csv", sep=";", decimal=".")